In [1]:
import numpy as np
import pandas as pd
import pickle


# EEGNet-specific imports
from EEGModels import DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix


In [2]:
data_type='mind-wandering'

In [3]:
def load_dataset_ctrvsone_and_transfer(user_split = True, runidx=1, data_type='mind-wandering', one_class='HT'):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    X_test = None
    y_test_medid = None
    y_test_subjid = None

    dtype_names = ["HT", "SNY", "VIP", "CTR"]

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = '../../iconip_data/{}/user_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)
    else:
        data_file_path = '../../iconip_data/{}/time_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        data_index_label_oneclass = dtype_names.index(one_class)
        data_index_label_ctr = dtype_names.index('CTR')
        transfer_index_labels = [x for x in list(set([0,1,2,3]).difference(set([dtype_names.index(one_class), 3]))) ]
        
        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']
        clf_config_subset_indices = np.where((y_train_medid == data_index_label_oneclass) | (y_train_medid == data_index_label_ctr))[0]
        X_train = X_train[(clf_config_subset_indices)]
        y_train_medid = y_train_medid[(clf_config_subset_indices)]
        y_train_subjid = y_train_subjid[clf_config_subset_indices]
        y_train_ts = y_train_ts[(clf_config_subset_indices)]

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']
        clf_config_subset_indices = np.where((y_val_medid == data_index_label_oneclass) | (y_val_medid == data_index_label_ctr))[0]
        X_val = X_val[(clf_config_subset_indices)]
        y_val_medid = y_val_medid[(clf_config_subset_indices)]
        y_val_subjid = y_val_subjid[(clf_config_subset_indices)]
        y_val_ts = y_val_ts[(clf_config_subset_indices)]
        
        # Pretrain test and transfer test set
        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']
        
        # transfer test contains CTR from test and other two classes from the test.
        ctr_othertwo_indices = np.where((y_test_medid == data_index_label_ctr) | (y_test_medid == transfer_index_labels[0]) | (y_test_medid == transfer_index_labels[1]))[0]
        X_test_transfer = X_test[(ctr_othertwo_indices)]
        y_test_transfer_medid = y_test_medid[(ctr_othertwo_indices)]
        y_test_transfer_subjid = y_test_subjid[(ctr_othertwo_indices)]
        
        # pretrain test filter CTR and one_class from the test.
        clf_config_subset_indices = np.where((y_test_medid == data_index_label_oneclass) | (y_test_medid == data_index_label_ctr))[0]
        X_test = X_test[(clf_config_subset_indices)]
        y_test_medid = y_test_medid[(clf_config_subset_indices)]
        y_test_subjid = y_test_subjid[(clf_config_subset_indices)]
        y_test_ts = y_test_ts[(clf_config_subset_indices)]

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid

In [4]:
def train_model(user_split, med_tech_clf, data_type, one_class):
    
    kernels, chans, samples = 1, 64, 2560
    
    split_strat = 'user' if user_split else 'time'
    model_results = pd.DataFrame()

    for runidx in range(1, 6):
        model_results['{}_Train_Acc'.format(runidx)] = None
        model_results['{}_Val_Acc'.format(runidx)] = None
        model_results['{}_Test_Acc'.format(runidx)] = None
        model_results['{}_Test_Transfer_Acc'.format(runidx)] = None
        model_results['{}_best_params'.format(runidx)] = None


    for runidx in range(1, 6):
        print("RUN ID: ", runidx)
        med_tech_clf = True
        
        X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_transfer, y_test_transfer_medid, y_test_transfer_subjid = load_dataset_ctrvsone_and_transfer(user_split=user_split, runidx=runidx, data_type=data_type, one_class=one_class)
        ## Convert data to binary classification task, i.e. meditation expert as class 0 vs control group as class 1
        y_train_medid = y_train_medid//3
        y_val_medid = y_val_medid//3
        y_test_medid = y_test_medid//3
        y_test_transfer_medid = y_test_transfer_medid//3

        best_clf_ours = None
        best_clf_val = 0

        #X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid
        if med_tech_clf:
            Y_train      = y_train_medid
            Y_validate   = y_val_medid
            Y_test       = y_test_medid
            Y_transfer   = y_test_transfer_medid
            ckpt_file_suffix = '{}_med_clf'.format(split_strat)
        else:
            Y_train      = y_train_subjid
            Y_validate   = y_val_subjid
            Y_test       = y_test_subjid
            Y_transfer   = y_test_transfer_subjid
            ckpt_file_suffix = '{}_subj_clf'.format(split_strat)
        nb_classes = np.unique(Y_train).shape[0]
        print(np.unique(Y_train), ": unique class labels.")
    
        ############################# EEGNet portion ##################################
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train)
        Y_validate   = np_utils.to_categorical(Y_validate)
        Y_test       = np_utils.to_categorical(Y_test)
        Y_transfer   = np_utils.to_categorical(Y_transfer)

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_val        = X_val.reshape(X_val.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        X_transfer   = X_transfer.reshape(X_transfer.shape[0], chans, samples, kernels)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')
        print(X_transfer.shape[0], 'transfer samples')
        print(nb_classes, " number of classes")
    
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = DeepConvNet(nb_classes = nb_classes, Chans = chans, Samples = samples, 
                    dropoutRate = 0.5)

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', 
                    metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()    

        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='./ckpt/deepconv/checkpoint_{}.h5'.format(ckpt_file_suffix), 
                                    verbose=1, save_best_only=True)
        
        ###############################################################################
        # if the classification task was imbalanced (significantly more trials in one
        # class versus the others) you can assign a weight to each class during 
        # optimization to balance it out. This data is approximately balanced so we 
        # don't need to do this, but is shown here for illustration/completeness. 
        ###############################################################################

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {cl: 1 for cl in range(nb_classes)}

        ################################################################################
        # fit the model. Due to very small sample sizes this can get
        # pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
        # Riemannian geometry classification (below)
        ################################################################################
        escallback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        fittedModel = model.fit(X_train, Y_train, batch_size = 128, epochs = 15, 
                                verbose = 2, validation_data=(X_val, Y_validate),
                                callbacks=[checkpointer, escallback], class_weight = class_weights)

        # load optimal weights
        model.load_weights('./ckpt/deepconv/checkpoint_{}.h5'.format(ckpt_file_suffix))
    
        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)

        probs_transfer = model.predict(X_transfer)
        preds_transfer = probs_transfer.argmax(axis = -1)
        
        # save preds on train and val set for error analysis
        preds_train_eegnet = model.predict(X_train)
        preds_train_eegnet = preds_train_eegnet.argmax(axis = -1)
        preds_val_eegnet = model.predict(X_val)
        preds_val_eegnet = preds_val_eegnet.argmax(axis = -1)
        
        
        acc_test         = np.mean(preds == Y_test.argmax(axis=-1))
        acc_transfer     = np.mean(preds_transfer == Y_transfer.argmax(axis=-1))
        print("DeepConv Classification accuracy test and transfer: %f %f" % (acc_test, acc_transfer))
        model_results.loc["DeepConv", ['{}_Train_Acc'.format(runidx), '{}_Val_Acc'.format(runidx), '{}_Test_Acc'.format(runidx), '{}_Test_Transfer_Acc'.format(runidx)]] = [preds_train_eegnet, preds_val_eegnet, acc_test, acc_transfer]
    if user_split:
        model_results.to_csv('./RESULTS_T5{}_MED_TransferPRED_{}_CTRvs{}_DeepConv.csv'.format('USER', data_type, one_class))
    else:
        model_results.to_csv('./RESULTS_T5_MED_TransferPRED_{}_CTRvs{}_DeepConv.csv'.format(data_type, one_class))

    return model_results

In [5]:
# time split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=False, med_tech_clf=True, data_type='mind-wandering', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (983, 64, 2560, 1)
983 train samples
348 test samples
511 transfer samples
2  number of classes


2022-06-23 15:14:34.489452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/15


2022-06-23 15:14:37.411171: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2093875200 exceeds 10% of free system memory.
2022-06-23 15:14:39.591706: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2093875200 exceeds 10% of free system memory.
2022-06-23 15:14:45.184761: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2093875200 exceeds 10% of free system memory.
2022-06-23 15:14:46.221842: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2093875200 exceeds 10% of free system memory.
2022-06-23 15:14:50.908078: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2093875200 exceeds 10% of free system memory.



Epoch 1: val_loss improved from inf to 1.04805, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 54s - loss: 3.2547 - accuracy: 0.5076 - val_loss: 1.0481 - val_accuracy: 0.5808 - 54s/epoch - 7s/step
Epoch 2/15

Epoch 2: val_loss improved from 1.04805 to 0.61808, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 49s - loss: 1.0677 - accuracy: 0.5778 - val_loss: 0.6181 - val_accuracy: 0.6587 - 49s/epoch - 6s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.61808
8/8 - 46s - loss: 0.5140 - accuracy: 0.7447 - val_loss: 0.8668 - val_accuracy: 0.6018 - 46s/epoch - 6s/step
Epoch 4/15

Epoch 4: val_loss improved from 0.61808 to 0.33302, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 48s - loss: 0.3836 - accuracy: 0.8210 - val_loss: 0.3330 - val_accuracy: 0.8593 - 48s/epoch - 6s/step
Epoch 5/15

Epoch 5: val_loss improved from 0.33302 to 0.17850, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 45s - loss: 0.2113 - accur

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (979, 64, 2560, 1)
979 train samples
347 test samples
512 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 4.63422, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 39s - loss: 3.4912 - accuracy: 0.5291 - val_loss: 4.6342 - val_accuracy: 0.5030 - 39s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss improved from 4.63422 to 0.60445, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 40s - loss: 1.5290 - accuracy: 0.5342 - val_loss: 0.6045 - val_accuracy: 0.6333 - 40s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.60445
8/8 - 41s - loss: 0.7851 - accuracy: 0.6292 - val_loss: 2.0146 - val_accuracy: 0.5636 - 41s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.60445
8/8 - 41s - loss: 0.7722 - accuracy: 0.6925 - val_loss: 1.7747 - val_accuracy: 0.5636 - 41s/epoch - 5s/step
Epoch 5/15

Epoch 5: val_loss did not improve from 0.60445
8/8 - 40s

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (979, 64, 2560, 1)
979 train samples
347 test samples
512 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 5.78998, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 40s - loss: 3.9880 - accuracy: 0.4944 - val_loss: 5.7900 - val_accuracy: 0.5121 - 40s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss improved from 5.78998 to 0.88598, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 41s - loss: 1.6918 - accuracy: 0.5730 - val_loss: 0.8860 - val_accuracy: 0.5364 - 41s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.88598
8/8 - 41s - loss: 0.9440 - accuracy: 0.7048 - val_loss: 2.3033 - val_accuracy: 0.5333 - 41s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss improved from 0.88598 to 0.71876, saving model to ./ckpt/deepconv/checkpoint_time_med_clf.h5
8/8 - 40s - loss: 1.0052 - accuracy: 0.7426 - val_loss: 0.7188 - val_accuracy: 0.5455 - 40s/epoch - 5s/step
Epoc

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [6]:
# User split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=True, med_tech_clf=True, data_type='mind-wandering', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (1044, 64, 2560, 1)
1044 train samples
320 test samples
469 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 0.91212, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 42s - loss: 3.0117 - accuracy: 0.5153 - val_loss: 0.9121 - val_accuracy: 0.6047 - 42s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss did not improve from 0.91212
9/9 - 35s - loss: 1.0658 - accuracy: 0.6216 - val_loss: 1.6061 - val_accuracy: 0.3455 - 35s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.91212
9/9 - 36s - loss: 0.4703 - accuracy: 0.8170 - val_loss: 3.4448 - val_accuracy: 0.4385 - 36s/epoch - 4s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.91212
9/9 - 36s - loss: 0.2350 - accuracy: 0.9090 - val_loss: 3.5470 - val_accuracy: 0.5083 - 36s/epoch - 4s/step
10/10 [==============================] - 2s 225ms/step
DeepConv Classification accuracy test and transfer: 0.631250 0.5

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1029, 64, 2560, 1)
1029 train samples
317 test samples
478 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 1.09181, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 37s - loss: 2.7869 - accuracy: 0.5326 - val_loss: 1.0918 - val_accuracy: 0.3323 - 37s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss did not improve from 1.09181
9/9 - 35s - loss: 0.9737 - accuracy: 0.7085 - val_loss: 3.7868 - val_accuracy: 0.3323 - 35s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 1.09181
9/9 - 35s - loss: 0.2930 - accuracy: 0.8834 - val_loss: 1.1188 - val_accuracy: 0.6113 - 35s/epoch - 4s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 1.09181
9/9 - 35s - loss: 0.0936 - accuracy: 0.9631 - val_loss: 5.6993 - val_accuracy: 0.5266 - 35s/epoch - 4s/step
10/10 [==============================] - 2s 183ms/step
DeepConv Classification accuracy test and transfer: 0.413249 0.569038
RUN I

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1019, 64, 2560, 1)
1019 train samples
331 test samples
473 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 2.55574, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
8/8 - 37s - loss: 2.6947 - accuracy: 0.5250 - val_loss: 2.5557 - val_accuracy: 0.4794 - 37s/epoch - 5s/step
Epoch 2/15

Epoch 2: val_loss improved from 2.55574 to 1.05514, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
8/8 - 36s - loss: 1.5379 - accuracy: 0.6084 - val_loss: 1.0551 - val_accuracy: 0.5746 - 36s/epoch - 5s/step
Epoch 3/15

Epoch 3: val_loss improved from 1.05514 to 0.94492, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
8/8 - 36s - loss: 0.7631 - accuracy: 0.6948 - val_loss: 0.9449 - val_accuracy: 0.5873 - 36s/epoch - 5s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.94492
8/8 - 38s - loss: 0.3963 - accuracy: 0.8145 - val_loss: 1.3952 - val_accuracy: 0.4889 - 38s/epoch - 5s/step
Ep

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1068, 64, 2560, 1)
1068 train samples
299 test samples
464 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 3.50617, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 39s - loss: 2.8279 - accuracy: 0.5328 - val_loss: 3.5062 - val_accuracy: 0.4879 - 39s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 3.50617 to 3.44397, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 39s - loss: 0.9667 - accuracy: 0.6386 - val_loss: 3.4440 - val_accuracy: 0.4879 - 39s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss improved from 3.44397 to 1.06508, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 36s - loss: 0.5623 - accuracy: 0.7079 - val_loss: 1.0651 - val_accuracy: 0.3668 - 36s/epoch - 4s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 1.06508
9/9 - 37s - loss: 0.3987 - accuracy: 0.8315 - val_loss: 1.1817 - val_accuracy: 0.4879 - 37s/epoch - 4s/step
Ep

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1032, 64, 2560, 1)
1032 train samples
306 test samples
474 transfer samples
2  number of classes
Epoch 1/15

Epoch 1: val_loss improved from inf to 2.02067, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 40s - loss: 2.1868 - accuracy: 0.5378 - val_loss: 2.0207 - val_accuracy: 0.4748 - 40s/epoch - 4s/step
Epoch 2/15

Epoch 2: val_loss improved from 2.02067 to 1.88737, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 36s - loss: 0.9497 - accuracy: 0.6172 - val_loss: 1.8874 - val_accuracy: 0.4748 - 36s/epoch - 4s/step
Epoch 3/15

Epoch 3: val_loss improved from 1.88737 to 0.65381, saving model to ./ckpt/deepconv/checkpoint_user_med_clf.h5
9/9 - 36s - loss: 0.7791 - accuracy: 0.6841 - val_loss: 0.6538 - val_accuracy: 0.7138 - 36s/epoch - 4s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.65381
9/9 - 35s - loss: 0.3254 - accuracy: 0.8934 - val_loss: 1.2597 - val_accuracy: 0.6509 - 35s/epoch - 4s/step
Ep

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [7]:
print("Done")

Done
